# 제출 양식대로 저장하기

In [1]:
# preprocessing
import numpy as np
import pandas as pd
import tqdm
from scipy import stats

# imputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# model learning
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
from sklearn.utils.class_weight import compute_sample_weight

# 평가 지표
from sklearn.metrics import mean_absolute_error

In [2]:
# load data
test_raw = pd.read_csv('../data/test.csv')
spring = pd.read_csv('../data/spring_test_result.csv')
summer = pd.read_csv('../data/summer_test_result.csv')
fall = pd.read_csv('../data/fall_test_result_new_m.csv')
winter = pd.read_csv('../data/winter_test_result_new_mice2.csv')
test_next = pd.read_csv('../data/test_yests.csv')
submit_spring = pd.read_excel('../submit/subminssionUser.xlsx', sheet_name = 'SPRING')
submit_summer = pd.read_excel('../submit/subminssionUser.xlsx', sheet_name = 'SUMMER')
submit_autumn = pd.read_excel('../submit/subminssionUser.xlsx', sheet_name = 'AUTUMN')
submit_winter = pd.read_excel('../submit/subminssionUser.xlsx', sheet_name = 'WINTER')

In [3]:
# 결합 전 변수 넣기
test_next['mmddhh'] = test_raw['surface_tp_test.mmddhh']

In [4]:
# left join
merge_spring = pd.merge(spring, test_next, how = 'left', on = ['stn', 'year', 'mm', 'dd', 'hh'])
merge_summer = pd.merge(summer, test_next, how = 'left', on = ['stn', 'year', 'mm', 'dd', 'hh'])
merge_autumn = pd.merge(fall, test_next, how = 'left', on = ['stn', 'mm', 'dd', 'hh'])
merge_winter = pd.merge(winter, test_next, how = 'left', on = ['stn', 'mm', 'dd', 'hh'])

In [5]:
# 새로운 파일에 넣을 값 지정하기
tmp_spring = pd.merge(merge_spring, submit_spring, how = 'right', left_on = ['stn', 'year', 'mmddhh'] , right_on = ['STN', 'YEAR', 'MMDDHH'])
tmp_summer = pd.merge(merge_summer, submit_summer, how = 'right', left_on = ['stn', 'year', 'mmddhh'] , right_on = ['STN', 'YEAR', 'MMDDHH'])
tmp_autumn = pd.merge(merge_autumn, submit_autumn, how = 'right', left_on = ['stn', 'year_x', 'mmddhh'] , right_on = ['STN', 'YEAR', 'MMDDHH'])
tmp_winter = pd.merge(merge_winter, submit_winter, how = 'right', left_on = ['stn', 'year_x', 'mmddhh'] , right_on = ['STN', 'YEAR', 'MMDDHH'])

In [6]:
# 필요한 값만 뽑기
tmp_spring['TS'] = tmp_spring['pred_ts']
tmp_summer['TS'] = tmp_summer['pred_ts']
tmp_autumn['TS'] = tmp_autumn['pred_ts']
tmp_winter['TS'] = tmp_winter['pred_ts']

In [7]:
# 제출 형식으로 저장
with pd.ExcelWriter('../submit/230175.xlsx') as writer:
    tmp_spring[['STN', 'YEAR', 'MMDDHH', 'TS']].to_excel(writer, sheet_name = 'SPRING', index = False)
    tmp_summer[['STN', 'YEAR', 'MMDDHH', 'TS']].to_excel(writer, sheet_name = 'SUMMER', index = False)
    tmp_autumn[['STN', 'YEAR', 'MMDDHH', 'TS']].to_excel(writer, sheet_name = 'AUTUMN', index = False)
    tmp_winter[['STN', 'YEAR', 'MMDDHH', 'TS']].to_excel(writer, sheet_name = 'WINTER', index = False)